This notebook converts diverse time series data for COVID-19 testing into a consistent BigQuery format
this is compatible with the table format dervied from the Johns Hopkins data (see the <a href="https://docs.google.com/document/d/1-EAMvCt2o3grhgRBF4gUXc6GvYLzaFxBbziPHrGLG6E/edit?hl=en#">doc</a> for details). This data is all updated daily.

*National Testing Data Time Series*	
* UK	https://github.com/emmadoughty/Daily_COVID-19/blob/master/COVID19_by_day.csv
* S Korea	https://github.com/jihoo-kim/Coronavirus-Dataset/blob/master/time.csv
* Italy	https://docs.google.com/spreadsheets/d/1iyvuTCaSq097WXwjkAMeiPjrGkNKav1XwS4J9abjjvk/edit#gid=0
	
*Regional*	
* US	https://docs.google.com/spreadsheets/u/1/d/e/2PACX-1vRwAqp96T9sYYq2-i7Tj0pvTf6XVHjDSMIKBdZHXiCGGdNC0ypEU9NbngS8mxea55JuCFuua1MUeOj5/pubhtml#

	
	

In [95]:
#create pandas data frame
import pandas
df = pandas.DataFrame(columns=['Province_State','Country_Region','Date','Type','Lat','Long','Val'])
index = 0

schema = [
  {
    "name": "Province_State",
    "mode": "nullable",
    "type": "STRING"
  },
  {
    "name": "Country_Region",
    "mode": "nullable",
    "type": "STRING"
  },
  {
    "name": "Lat",
    "mode": "nullable",
    "type": "FLOAT"
  },
  {
    "name": "Long",
    "mode": "nullable",
    "type": "FLOAT"
  }, 
  {
    "name": "Date",
    "mode": "nullable",
    "type": "DATE"
  },
  {
    "name": "Val",
    "mode": "nullable",
    "type": "INTEGER"
  },
  {
    "name": "Type",
    "mode": "nullable",
    "type": "STRING"
  } 
]

In [63]:
PROJECT = 'big-query-tests-197122'
BUCKET = 'ronbodkin-covid-etl'

In [158]:
%%bash

bq_safe_mk() {
    dataset=$1
    exists=$(bq ls -d | grep -w $dataset)
    if [ -n "$exists" ]; then
       echo "Not creating $dataset since it already exists"
    else
       echo "Creating $dataset"
       bq mk $dataset
    fi
}

bq_safe_mk advdata

Not creating advdata since it already exists


In [84]:
#UK data

In [159]:
%%bash
rm -rf COVID19_by_day.csv
wget --quiet https://raw.githubusercontent.com/emmadoughty/Daily_COVID-19/master/COVID19_by_day.csv

In [18]:
!ls *.csv

COVID19_by_day.csv


In [160]:
import dateutil.parser

ifp = open('COVID19_by_day.csv')
count = 0

for line in ifp: 
    count += 1
    if (count > 1):
        cols = line.strip().split(',')
        df.loc[index] = pandas.Series({'Province_State':'UK', 'Country_Region':'United Kingdom',\
                                       'Lat':55.3781, 'Long':-3.436, \
                                         'Date':dateutil.parser.parse(cols[0], dayfirst=True), \
                                         'Val':cols[4], 'Type': 'Tests'})
        index += 1

In [161]:
print(df[0:3])

  Province_State  Country_Region       Date   Type      Lat   Long Val
0             UK  United Kingdom 2020-01-25  Tests  55.3781 -3.436  31
1             UK  United Kingdom 2020-01-26  Tests  55.3781 -3.436  52
2             UK  United Kingdom 2020-01-27  Tests  55.3781 -3.436  73


In [ ]:
#S Korea Data

In [162]:
%%bash
rm -rf time.csv
wget --quiet https://raw.githubusercontent.com/jihoo-kim/Coronavirus-Dataset/master/time.csv

In [163]:
import dateutil.parser

ifp = open('time.csv')
count = 0

for line in ifp: 
    count += 1
    cols = line.strip().split(',')
    if (count == 1):
        header = cols
    else:
        df.loc[index] = pandas.Series({'Province_State':None, 'Country_Region':'Korea, South',\
                                       'Lat':36.0, 'Long':128.0, \
                                         'Date':dateutil.parser.parse(cols[0]), \
                                         'Val':cols[2], 'Type': 'Tests'})
        index += 1
        for i in range (7, len(cols)):
            df.loc[index] = pandas.Series({'Province_State':header[i], 'Country_Region':'Korea, South',\
                                       'Lat':36.0, 'Long':128.0, \
                                         'Date':dateutil.parser.parse(cols[0]), \
                                         'Val':cols[i], 'Type': 'Confirmed'})
            index += 1

In [164]:
df

,Province_State,Country_Region,Date,Type,Lat,Long,Val
0,UK,United Kingdom,2020-01-25,Tests,55.3781,-3.436,31
1,UK,United Kingdom,2020-01-26,Tests,55.3781,-3.436,52
2,UK,United Kingdom,2020-01-27,Tests,55.3781,-3.436,73
3,UK,United Kingdom,2020-01-28,Tests,55.3781,-3.436,97
4,UK,United Kingdom,2020-01-29,Tests,55.3781,-3.436,130
...,...,...,...,...,...,...,...
2643,Jeollabuk-do,"Korea, South",2020-03-12,Confirmed,36.0000,128.000,7
2644,Jeollanam-do,"Korea, South",2020-03-12,Confirmed,36.0000,128.000,4
2645,Gyeongsangbuk-do,"Korea, South",2020-03-12,Confirmed,36.0000,128.000,1143
2646,Gyeongsangnam-do,"Korea, South",2020-03-12,Confirmed,36.0000,128.000,85


In [ ]:
#US data

In [166]:
%%bash
rm -rf states_daily_4pm_et.csv
wget --quiet https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/states_daily_4pm_et.csv

In [167]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
#as of today West Virginia has no cases so we have to add it to the data set

In [168]:
%%bigquery usgeo

SELECT DISTINCT Province_State, Lat, Long
FROM advdata.covid_19_stats
WHERE Country_Region = 'US'
UNION ALL
SELECT 'West Virginia', 38.5976, -80.4549

In [169]:
import dateutil.parser

ifp = open('states_daily_4pm_et.csv')
count = 0

for line in ifp: 
    count += 1
    cols = line.strip().split(',')
    if (count == 1):
        assert(cols[0]=='date')
        assert(cols[1]=='state')
        assert(cols[6]=='total')
    else:
        state = abbrev_us_state[cols[1]]
        r = usgeo.set_index('Province_State').loc[state]
        df.loc[index] = pandas.Series({'Province_State':state, 'Country_Region':'Korea, South',\
                                       'Lat':r.Lat, 'Long':r.Long, \
                                         'Date':dateutil.parser.isoparse(cols[0]), \
                                         'Val':cols[6], 'Type': 'Tests'})
        index += 1


In [117]:
#Italy

In [53]:
#to convert PDF's you have to install the poppler package - this is using conda
!pip install pdf2image
!conda install -c conda-forge -y poppler

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - poppler


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py37hc8dfbb8_1         149 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_0         3.0 MB  conda-forge
    openjpeg-2.3.1             |       h981e76c_3         475 KB  conda-forge
    poppler-0.84.0             |       h9584818_0        12.3 MB  conda-forge
    poppler-data-0.4.9         |                1         3.4 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        19.3 MB

The following NEW packages will be INSTALLED:

  openjpeg           conda-forge/linux-64::openjpeg-2.3.1-h981e76c_3
  poppler        

In [4]:
!pip install gcloud
!pip install google-colab

     |████████████████████████████████| 72 kB 1.2 MB/s eta 0:00:011
     |████████████████████████████████| 64 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 104 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 758 kB 19.8 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 29.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 7.7 MB/s  eta 0:00:01
     |████████████████████████████████| 484 kB 45.5 MB/s eta 0:00:01
     |████████████████████████████████| 245 kB 44.9 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 75.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.6 MB/s  eta 0:00:01
  Created wheel for google-colab: filename=google_colab-1.0.0-py2.py3-none-any.whl size=102289 sha256=4efab7e94b4e500d18730da8cdf1b4c4d7ecb1bdd703346ba876a6632455b1c6
  Stored in directory: /home/jupyter/.cache/pip/wheels/f6/3b/58/f34ea9045a7c6

In [ ]:
from google.colab import files
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
from IPython.display import display
import json
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import time
from tqdm import tqdm

def authenticate():
  """Authenticates using a service account private key file.
  
  Returns:
    Authenticated session.
  """
  uploaded = files.upload()
  filename = list(uploaded)[0]
  service_account_key_dict = json.loads(uploaded[filename].decode('utf-8'))
  # Scoped credentials
  credentials = service_account.Credentials.from_service_account_info(
      service_account_key_dict,
      scopes=['https://www.googleapis.com/auth/cloud-platform'])
  sess = AuthorizedSession(credentials)
  return sess

authenticate()

In [ ]:
#parse from daily situation updates
#format is http://www.salute.gov.it/imgs/C_17_pagineAree_5351_2*X_file.pdf
import pytz
import dateutil
import PIL
import tempfile

from datetime import datetime, date, timedelta
from string import Formatter
from urllib.request import urlopen
from pdf2image import convert_from_bytes
from gcloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import os

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


first_day = date(2020,3,8)
last_day = datetime.now(pytz.timezone("CET")).date()
index = 0

for day in daterange(first_day, last_day):
    uri = "http://www.salute.gov.it/imgs/C_17_pagineAree_5351_{}_file.pdf".format(index);
    print(day.strftime("%Y-%m-%d"))
    print(uri)
    
    #download and convert to PNG
    stream = urlopen(uri).read()
    fp = tempfile.NamedTemporaryFile(suffix='.png')
    fp.close()
    print(fp.name)
    images = convert_from_bytes(stream)
    images[0].save(fp.name)
    
    credentials_dict = {
        'type': 'service_account',
        'client_id': os.environ['BACKUP_CLIENT_ID'],
        'client_email': os.environ['BACKUP_CLIENT_EMAIL'],
        'private_key_id': os.environ['BACKUP_PRIVATE_KEY_ID'],
        'private_key': os.environ['BACKUP_PRIVATE_KEY'],
    }
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(
        credentials_dict
    )
    client = storage.Client(credentials=credentials, project=PROJECT)
    bucket = client.get_bucket(BUCKET)
    blob = bucket.blob(fp.name)
    blob.upload_from_filename(fp.name)
    
    index += 2
                       

In [155]:
#not working yet
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SHEET_ID = '1iyvuTCaSq097WXwjkAMeiPjrGkNKav1XwS4J9abjjvk'
RANGE_ID = 'covid19!B1:T999'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SHEET_ID,
                            range=RANGE_ID).execute()
values = result.get('values', [])


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=236004641607-3iq363181f2eki2qmslrm6n98s6v1r9n.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A45155%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=un8CnEZUQO6xJ0ujC9bGVxeyGsnPpt&access_type=offline


KeyboardInterrupt: 

In [83]:
#export to BQ
#!pip install pandas-gbq -U

In [171]:
import pandas_gbq
pandas_gbq.to_gbq(df, 'advdata.tests', 'big-query-tests-197122', if_exists='replace')

1it [00:03,  3.47s/it]
